<a href="https://colab.research.google.com/github/maninog/langchain/blob/main/LangChain_Prompt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LangChain Prompt

## README
- author: [Masumi Morishige](https://twitter.com/masumi_creator)
- created_at: 2023-05-02
- updated_at: 2023-05-06

### 実行方法
1. OpenAIのAPIキーを発行
2. `os.environ["OPENAI_API_KEY"] = "..."`の`""`の中にご自身のAPIキーを代入
3. 「ランタイム > すべてのセルを実行」を実行

### 参考情報
- Zenn: [LangChain Promptとは？【Templates・Example Selectors・Output Parsers】](https://zenn.dev/umi_mori/books/prompt-engineer/viewer/langchain_prompt)
- YouTube: [LangChain Promptとは？【Templates / Example Selectors / Output Parsers】](https://youtu.be/N7IKDNCcUYA)

### OpenAI APIの発行方法

[<img src="https://img.youtube.com/vi/frpsKLNW1q4/maxresdefault.jpg" width="600px">](https://youtu.be/frpsKLNW1q4)

[【エンジニア向け】OpenAIのAPI連携方法【環境構築 + GASによるGoogle Documentへの組み込み】](https://youtu.be/frpsKLNW1q4)

## 環境構築

In [ ]:
!pip3 install langchain openai

In [ ]:
import os

#TODO: APIキーの登録が必要
os.environ["OPENAI_API_KEY"] = "..."

## 1. Prompt Templateの使い方

In [ ]:
from langchain import PromptTemplate
from langchain.llms import OpenAI

template = """
{subject}について30文字で教えて。
"""

prompt = PromptTemplate(
    input_variables=["subject"],
    template=template,
)
prompt_text = prompt.format(subject="ITエンジニア")
print(prompt_text)

llm = OpenAI(model_name="text-davinci-003")
print(llm(prompt_text))

In [ ]:
from langchain import PromptTemplate
from langchain import FewShotPromptTemplate
from langchain.llms import OpenAI

examples = [
    {"fruit": "りんご", "color": "赤"},
    {"fruit": "キウイ", "color": "緑"},
    {"fruit": "ぶどう", "color": "紫"},
]

example_formatter_template = """
フルーツ: {fruit}
色: {color}\n
"""
example_prompt = PromptTemplate(
    input_variables=["fruit", "color"],
    template=example_formatter_template,
)

few_shot_prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix="フルーツの色を漢字で教えて。",
    suffix="フルーツ: {input}\n色:",
    input_variables=["input"],
    example_separator="\n\n",

)

prompt_text = few_shot_prompt.format(input="オレンジ")
print(prompt_text)

llm = OpenAI(model_name="text-davinci-003")
print(llm(prompt_text))

In [ ]:
from langchain.prompts import (
    ChatPromptTemplate,
    PromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)
from langchain.chat_models import ChatOpenAI

system_template="あなたは、質問者からの質問を{language}で回答するAIです。"
human_template="質問者：{question}"
system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])
prompt_message_list = chat_prompt.format_prompt(language="日本語", question="ITエンジニアについて30文字で教えて。").to_messages()

print(prompt_message_list)

chat = ChatOpenAI(model_name="gpt-3.5-turbo")
chat(prompt_message_list)

In [ ]:
from langchain.prompts import load_prompt

prompt = load_prompt("lc://prompts/conversation/prompt.json")
prompt_text = prompt.format(history="こんにちは、ますみです。", input="ITエンジニアについて30文字で教えて。")
print(prompt_text)

## 2. Example Selectorsの使い方

In [ ]:
from langchain.prompts.example_selector.base import BaseExampleSelector
from typing import Dict, List
import numpy as np


class CustomExampleSelector(BaseExampleSelector):

    def __init__(self, examples: List[Dict[str, str]]):
        self.examples = examples

    def add_example(self, example: Dict[str, str]) -> None:
        """新しい教師データを格納するめの関数"""
        self.examples.append(example)

    def select_examples(self, input_variables: Dict[str, str]) -> List[dict]:
        """教師データを選択するための関数"""
        return np.random.choice(self.examples, size=2, replace=False)

examples = [
    {"fruit": "りんご", "color": "赤"},
    {"fruit": "キウイ", "color": "緑"},
    {"fruit": "ぶどう", "color": "紫"},
]

# 教師データの登録
example_selector = CustomExampleSelector(examples)

# 教師データの出力
print(example_selector.examples)

# 教師データの追加
example_selector.add_example({"fruit": "オレンジ", "color": "橙"})
print(example_selector.examples)

# 教師データの選択
print(example_selector.select_examples({"fruit": "fruit"}))

## 3. Output Parsersの使い方

In [ ]:
from langchain.output_parsers import CommaSeparatedListOutputParser
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI

output_parser = CommaSeparatedListOutputParser()
format_instructions = output_parser.get_format_instructions()
prompt = PromptTemplate(
    template="List five {subject}.\n{format_instructions}",
    input_variables=["subject"],
    partial_variables={"format_instructions": format_instructions}
)

llm = OpenAI(model_name="text-davinci-003")
_input = prompt.format(subject="Programming Language")
output = llm(_input)
output_parser.parse(output)

In [ ]:
from typing import List
from pydantic import BaseModel, Field

from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI
from langchain.output_parsers import PydanticOutputParser

class Job(BaseModel):
    name: str = Field(description="Name of the job")
    skill_list: List[str] = Field(description="List of skills required for that job")

parser = PydanticOutputParser(pydantic_object=Job)
prompt = PromptTemplate(
    template="{query}\n\n{format_instructions}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()}
)
_input = prompt.format_prompt(query="Tell me the skills required for frontend engineer.")
print(_input)

llm = OpenAI(model_name="text-davinci-003")
output = llm(_input.to_string())
print(output)

parser.parse(output)

In [ ]:
from typing import List

from pydantic import BaseModel, Field

from langchain.chat_models import ChatOpenAI
from langchain.output_parsers import (
    PydanticOutputParser,
    OutputFixingParser,
)

class Job(BaseModel):
    name: str = Field(description="Name of the job")
    skill_list: List[str] = Field(description="List of skills required for that job")

misformatted = "{'name': 'Frontend Engineer', 'skill_list': ['HTML', 'CSS', 'JS', 'TS']}"

parser = PydanticOutputParser(pydantic_object=Job)

new_parser = OutputFixingParser.from_llm(parser=parser, llm=ChatOpenAI(model_name="gpt-3.5-turbo"))
new_parser.parse(misformatted)